In [1]:
import sympy as sp
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(threshold=np.inf)

Definición de particiones y tamaños del sistema

In [2]:
dx=.2
dy=.2
min_x=0
max_x=1
min_y=0
max_y=1
#--------------------------------
xi = np.arange(dx,max_x + dx,dx)
yi = np.arange(dy,max_y + dy,dy)
lenx = int((max_x-min_x)/dx)
leny = int((max_y-min_y)/dy)
M = np.zeros((lenx,leny))#determina el mallado segun sus limites dividido en su partición
print(M)

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


Definición de condiciones de borde. Nótese que el 1 en la definición representa a la última fila/columna de la matriz

In [16]:
M[0,:] = 0       # FILA ARRIBA f(x,0) = condición
M[leny-1,:] = 0  # FILA ABAJO  f(x,1) = condicion
M[:,0] = 0       # COLUMNA IZQ f(0,y) = condicion
M[:,lenx-1] = 1  # COLUMNA DER f(1,y) = condicion
print(M)
#limites de iteración
def lim(i,j,a,b):
    if(M[i][j]!=0):
        return a
    else:
        return b
#lim = lambda x: 1 if(x!=0) else 0
lim_inf_x = lim(0,1,1,0)
lim_sup_x = lim(lenx-1,1,lenx-1,lenx)
lim_inf_y = lim(0,0,1,0)
lim_sup_y = lim(1,leny-1,leny-1,leny)

print(lim_inf_x)
print(lim_sup_x)
print(lim_inf_y)
print(lim_sup_y)

[[0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]]
0
5
0
4


In [33]:
#truco: tener un vector de nodos, para ubicar los f en las ecuaciones correspondientes.
lenx = lim_sup_x - lim_inf_x
leny = lim_sup_y - lim_inf_y
N = np.zeros((leny,lenx))
nodo=0
for i in range(leny):
    for j in range(lenx):
        N[i,j]=nodo
        nodo +=1
N = N.astype(int)
N

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14]])

In [35]:
def D2fdx(eq,b,i,j):
    if(i==j==0):
        eq[N[i,j]]   += -2*M[i,j]/dx**2
        if(M[i,j]==0):
            eq[N[i,j]]   += -2*1/dx**2
    if(i+1<lenx):
        eq[N[i+1,j]] += 1/dx**2
    else:
        b[N[i,j]] -= M[i+1,j]/dx**2
    if(i-1>=0):
        eq[N[i-1,j]] += 1/dx**2
    else:
        b[N[i,j]] -= M[i-1,j]/dx**2
    return eq,b

def D2fdy(eq,b,i,j):
    if(i==j==0):
        eq[N[i,j]]   += -2*M[i,j]/dy**2
        if(M[i,j]==0):
            eq[N[i,j]]   += -2*1/dy**2
    else:
        eq[N[i,j]]   += -2*1/dy**2
    if(j+1<leny):
        eq[N[i,j+1]] += 1/dy**2
    else:
        b[N[i,j]] -= M[i,j+1]/dy**2
    if(j-1>=0):
        eq[N[i,j]] += 1/dy**2
    else:
        b[N[i,j]] -= M[i,j-1]/dy**2
    return eq,b

def f(b,i,j):
    ## la funcion es x
    b[N[i,j]] += xi[i]
    return b


eqs = np.zeros(lenx*leny)
b  =  np.zeros(lenx*leny)
for j in range(lim_inf_x,lim_sup_x):
    for i in range(lim_inf_y,lim_sup_y):
        eq = np.zeros(lenx*leny)
        
        # Cálculo de valores de las ecuaciones de los nodos.
        b = f(b,i,j)
        eq,b = D2fdx(eq,b,i,j)
        eq,b = D2fdy(eq,b,i,j)
        
        #agregar fila a la matriz eqs a resolver
        if(i==0 and j==0):
            eqs = eq
        else:
            eqs = np.vstack([eqs,eq])
        
print(eqs[0])
print(b[0])
#np.linalg.solve(eqs,b)

IndexError: index 5 is out of bounds for axis 0 with size 5